# Post processing of Extracted Radiomic Features 

In [1]:
# Intention Hassan transform: Render feature independent of gray level discretization.

In [2]:
import sys
sys.path.append('./../utils/')

import os
import re

import numpy as np
import pandas as pd

import ioutil

from transforms import Whitening
from processing import PostProcessor

from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
true_index = np.load('./../../../data_source/patient_id.npy')

In [4]:
# Globals.
FILTER_TYPE = 'original'

## Process Raw Features

In [5]:
ct_feature_files = [
    './../../../data_source/radiomic_features/no_filter/ct_32_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/ct_64_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/ct_128_no_filter_config.csv',
]
pet_feature_files = [
    './../../../data_source/radiomic_features/no_filter/pet_32_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/pet_64_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/pet_128_no_filter_config.csv'
]

In [6]:
ct_processor = PostProcessor(
    path_to_features=ct_feature_files, 
    indices=true_index, 
    filter_type=FILTER_TYPE, 
    error_dir='./../../../data_source/to_analysis/no_filter/errors/'
)

In [7]:
ct_processor.process(drop_redundant=True, drop_missing=True)
ct_processor.to_file(
    path_to_dir='./../../../data_source/to_analysis/no_filter/'
)

In [8]:
pet_processor = PostProcessor(
    path_to_features=pet_feature_files, 
    indices=true_index, 
    filter_type=FILTER_TYPE, 
    error_dir='./../../../data_source/to_analysis/no_filter/errors/'
)

In [9]:
pet_processor.process(drop_redundant=True, drop_missing=True)
pet_processor.to_file(
    path_to_dir='./../../../data_source/to_analysis/no_filter/'
)

In [10]:
ct_features = ct_processor.concatenated
pet_features = pet_processor.concatenated
np.shape(ct_features), np.shape(pet_features)

((198, 276), (198, 276))

## Concatenate Features

In [11]:
shape_feats = pd.read_csv('./../../../data_source/to_analysis/radiomics_shape.csv', index_col=0)
clinical = pd.read_csv('./../../../data_source/to_analysis/clinical_params.csv', index_col=0)
pet_params = pd.read_csv('./../../../data_source/to_analysis/pet_params.csv', index_col=0)

In [12]:
ct_features.columns = ['CT_{}'.format(col) for col in ct_features.columns]
pet_features.columns = ['PET_{}'.format(col) for col in pet_features.columns]

In [13]:
X = pd.concat((shape_feats, clinical, ct_features, pet_features, pet_params), axis=1)
np.shape(X)

(198, 610)

In [14]:
X.to_csv('./../../../data_source/to_analysis/no_filter_concat.csv')